In [16]:
import tensorflow as tf 
from tensorflow import keras 
import numpy as np

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [17]:
tf.__version__

'2.2.0'

In [18]:
keras.__version__

'2.3.0-tf'

In [19]:
import pandas as pd
pd_test = pd.read_csv('digit-recognizer/test.csv')
pd_train = pd.read_csv('digit-recognizer/train.csv')
pd_sample_submission = pd.read_csv('digit-recognizer/sample_submission.csv')

In [20]:
from sklearn.model_selection import train_test_split

y = pd_train.pop('label')
X = pd_train

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.01)


In [21]:
X_train

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
25371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24759,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16680,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18751,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26857,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16768,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
from scipy.ndimage.interpolation import shift
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])
X_train_augmented = [image for image in X_train.to_numpy()]
y_train_augmented = [label for label in y_train.to_numpy()]

for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train.to_numpy(), y_train.to_numpy()):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

In [33]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28*28]),
    keras.layers.Dense(450, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(225, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(110, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),

    keras.layers.Dense(10, activation="softmax")
])


In [34]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])


In [35]:
X_train_augmented

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [37]:

history = model.fit(X_train_augmented, y_train_augmented, epochs=15,
                    validation_data=(X_test.to_numpy(), y_test.to_numpy()))


Epoch 1/15
6497/6497 [==============================] - 8s 1ms/step - loss: 1.3936 - accuracy: 0.9066 - val_loss: 0.3015 - val_accuracy: 0.9381
Epoch 2/15
6497/6497 [==============================] - 8s 1ms/step - loss: 0.2065 - accuracy: 0.9520 - val_loss: 0.2077 - val_accuracy: 0.9571
Epoch 3/15
6497/6497 [==============================] - 8s 1ms/step - loss: 0.1362 - accuracy: 0.9641 - val_loss: 0.2184 - val_accuracy: 0.9452
Epoch 4/15
6497/6497 [==============================] - 8s 1ms/step - loss: 0.1021 - accuracy: 0.9710 - val_loss: 0.1975 - val_accuracy: 0.9714
Epoch 5/15
6497/6497 [==============================] - 8s 1ms/step - loss: 0.0817 - accuracy: 0.9759 - val_loss: 0.1552 - val_accuracy: 0.9667
Epoch 6/15
6497/6497 [==============================] - 8s 1ms/step - loss: 0.0675 - accuracy: 0.9797 - val_loss: 0.1433 - val_accuracy: 0.9690
Epoch 7/15
6497/6497 [==============================] - 8s 1ms/step - loss: 0.0564 - accuracy: 0.9826 - val_loss: 0.1342 - val_accuracy:

In [38]:
pd_test.to_numpy()



array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [39]:
predictions = model.predict(pd_test.to_numpy())


In [40]:
classes = np.argmax(predictions, axis = 1)


In [41]:
classes

array([2, 0, 9, ..., 3, 9, 2])

In [42]:
pd_sample_submission = pd_sample_submission.assign(Label=classes)


In [43]:
pd_sample_submission.to_csv(path_or_buf = './file.csv', index=False)